In [14]:
import os

In [15]:
%pwd

'c:\\Users\\VAIO\\OneDrive\\Desktop\\Chiken_disease_classification_project\\Chiken_disease_classification'

In [16]:
os.chdir("../")

In [25]:
%pwd

'c:\\Users\\VAIO\\OneDrive\\Desktop\\Chiken_disease_classification_project'

In [26]:
%cd Chiken_disease_classification/

c:\Users\VAIO\OneDrive\Desktop\Chiken_disease_classification_project\Chiken_disease_classification


In [28]:
%ls

 Volume in drive C has no label.
 Volume Serial Number is E0B8-81FC

 Directory of c:\Users\VAIO\OneDrive\Desktop\Chiken_disease_classification_project\Chiken_disease_classification

03/17/2025  08:56 AM    <DIR>          .
03/17/2025  08:56 AM    <DIR>          ..
03/10/2025  08:38 AM    <DIR>          .github
03/10/2025  07:59 AM             3,586 .gitignore
03/17/2025  08:57 AM    <DIR>          artifacts
03/13/2025  10:38 AM    <DIR>          config
03/10/2025  09:50 AM                 0 dvc.yaml
03/13/2025  09:03 AM    <DIR>          logs
03/13/2025  10:38 AM               121 main.py
03/13/2025  10:38 AM                18 params.yaml
03/13/2025  10:38 AM               276 README.md
03/12/2025  09:10 AM               154 requirements.txt
03/13/2025  10:38 AM    <DIR>          research
03/12/2025  08:42 AM               284 setup.py
03/10/2025  08:38 AM    <DIR>          src
03/10/2025  09:50 AM             1,325 template.py
03/10/2025  09:51 AM    <DIR>          templates
        

In [54]:
# Entity is nothing but it is the return type of the function
from dataclasses import dataclass
from pathlib import Path

In [55]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path


In [56]:
# To read the yaml file you first need to mention it in the constant directory (constructor)
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml,create_directories


In [57]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir  
        )
        return data_ingestion_config

In [58]:
import os
import urllib.request as request
import zipfile
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size



In [59]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename , headers= request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size : {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        """
        zip_file_path:str
        Extracts the zip fle into data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)
        


In [60]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-17 09:11:21,411:INFO:common:yaml file: config\config.yaml loaded successfully]
[2025-03-17 09:11:21,414:INFO:common:yaml file: params.yaml loaded successfully]
[2025-03-17 09:11:21,416:INFO:common:created directory at : artifacts]
[2025-03-17 09:11:21,419:INFO:common:created directory at : artifacts/data_ingestion]
[2025-03-17 09:11:32,492:INFO:3564325191:artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 78B4:27F8C2:429174:55DEAF:67D849AA
Accept-Ranges: bytes
Date: Mon, 17 Mar 2025 16:11:23 GMT
Via: 1.1 varnish
X-Served-By: cache-fjr990031-FJR
X-Cache: MISS
X-Cache